In [60]:
import ctypes
import multiprocessing as mp
import sys
from time import perf_counter as time
import numpy as np
from PIL import Image
def init(shared_arr_):
    global shared_arr
    shared_arr = shared_arr_
def tonumpyarray(mp_arr):
    return np.frombuffer(mp_arr, dtype='float64')

def reduce_step(args):
    batch_index, batch_index_offset, wtf, image_shape = args

    # this recovers the whole array
        # B x W x H x C
    arr = tonumpyarray(shared_arr).reshape((-1,) + image_shape)
    # We then add the all the offset numbers
    if batch_index >= len(arr) or batch_index_offset >= len(arr):
        pass
    else:
        arr[batch_index] += arr[batch_index_offset]
    


In [61]:
if __name__ == '__main__':

    
    n_processes = 1
    chunk = 2
    # Create shared array
    #data = np.load(sys.argv[1]) # B x W x H x C
    B = 10
    
    W = 100
    H = 100
    C = 3
    data = np.random.rand(B,W,H,C)
    #data = np.array([[1,2,3],[2,3,4],[4,5,6], [4,5,6]])
    #print(data.mean(axis=1))
    #B,W,H,C = data.shape
    B=len(data)
    elemshape = data.shape[1:] # W x H x C
    n_elements = np.prod(data.shape) # B * W * H * C
    shared_arr = mp.RawArray(ctypes.c_float, data.size) 
    arr = tonumpyarray(shared_arr).reshape(data.shape)
    np.copyto(arr, data)
    #del data
    
    # Run parallel sum
    t = time()
    pool = mp.Pool(n_processes, initializer=init, initargs=(shared_arr,))
    # Change the code below to compute the full reduction and the mean
    # ---------------------------8<---------------------------
    # So we are iterating over the batch dimension by the chunk, first we sum the neighbours,
    # I suppose next time we should like jump
    chunk = 1
    #offset = 2**n
    #skip = 2**(n+1)
    n = 0
    while True:
        offset = 2**n
        skip = 2**(n+1)

        pool.map(reduce_step, [(i, i + offset, 1, elemshape) for i in range(0, B, skip)], chunksize=1)
        n += 1
        if offset > B - 1:
            break

        
    
    #pool.map(reduce_step, [(i, i + chunk, 1, elemshape) for i in range(0, len(arr), chunk)], chunksize=1)
    # Write output
    print(time() - t)
    final_image = arr[0]/len(arr)
    # final_image /= len(arr) # For mean
    Image.fromarray(
    (255 * final_image.astype(float)).astype('uint8')
    ).save('result.png')

ValueError: cannot reshape array of size 150000 into shape (10,100,100,3)

In [56]:
final_image

array([[[0.42434245, 0.5433982 , 0.53760624],
        [0.45988584, 0.42563424, 0.51566964],
        [0.6317546 , 0.4399337 , 0.6418253 ],
        ...,
        [0.5026754 , 0.5552681 , 0.39095256],
        [0.3671933 , 0.52587605, 0.5613812 ],
        [0.54275787, 0.49280256, 0.4119298 ]],

       [[0.50767946, 0.31944606, 0.4651808 ],
        [0.59431976, 0.44064346, 0.42584592],
        [0.60578537, 0.47848693, 0.57131994],
        ...,
        [0.5129062 , 0.4210247 , 0.7129548 ],
        [0.5779824 , 0.45330754, 0.5830502 ],
        [0.5404177 , 0.57941526, 0.38099027]],

       [[0.6033215 , 0.57250595, 0.5248116 ],
        [0.5366856 , 0.30602995, 0.5154694 ],
        [0.48013878, 0.5151455 , 0.51328564],
        ...,
        [0.5138394 , 0.363814  , 0.48694506],
        [0.5720178 , 0.5215101 , 0.42189693],
        [0.59448946, 0.48579755, 0.46629715]],

       ...,

       [[0.5502822 , 0.4970134 , 0.45240474],
        [0.56438315, 0.48255554, 0.4808291 ],
        [0.62244856, 0

In [57]:
data.mean(axis=0)

array([[[0.42434245, 0.54339821, 0.53760625],
        [0.45988586, 0.42563424, 0.51566965],
        [0.63175458, 0.43993366, 0.64182532],
        ...,
        [0.50267541, 0.55526808, 0.39095256],
        [0.36719332, 0.52587605, 0.56138125],
        [0.54275783, 0.49280255, 0.41192977]],

       [[0.50767948, 0.31944608, 0.46518083],
        [0.59431979, 0.44064347, 0.42584593],
        [0.60578531, 0.47848688, 0.57131994],
        ...,
        [0.51290614, 0.4210247 , 0.7129548 ],
        [0.57798241, 0.45330755, 0.5830502 ],
        [0.54041767, 0.5794153 , 0.38099029]],

       [[0.60332149, 0.57250593, 0.52481166],
        [0.53668557, 0.30602994, 0.51546932],
        [0.48013874, 0.51514549, 0.51328565],
        ...,
        [0.5138394 , 0.36381398, 0.48694508],
        [0.57201777, 0.52151011, 0.42189693],
        [0.59448951, 0.48579755, 0.46629715]],

       ...,

       [[0.55028219, 0.49701338, 0.45240473],
        [0.56438311, 0.48255553, 0.48082909],
        [0.62244857, 0